In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import tensorflow as tf
print(tf.__version__)
import keras
print(keras.__version__)

/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/sunxin/miniconda3/envs/qinhanmin-test/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/sunxin/minico

1.13.2
2.2.4


Using TensorFlow backend.


In [2]:
import keras
from keras.datasets import mnist
from keras import models
from keras import layers

In [3]:
batch_size = 128
num_classes = 5
epochs = 5

# input image dimensions
img_rows, img_cols = 28, 28
# number of convolutional filters to use
filters = 32
# size of pooling area for max pooling
pool_size = 2
# convolution kernel size
kernel_size = 3

input_shape = (img_rows, img_cols, 1)

In [4]:
# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# create two datasets one with digits below 5 and one with 5 and above
x_train_lt5 = x_train[y_train < 5]
y_train_lt5 = y_train[y_train < 5]
x_test_lt5 = x_test[y_test < 5]
y_test_lt5 = y_test[y_test < 5]

x_train_gte5 = x_train[y_train >= 5]
y_train_gte5 = y_train[y_train >= 5] - 5
x_test_gte5 = x_test[y_test >= 5]
y_test_gte5 = y_test[y_test >= 5] - 5

In [5]:
# define two groups of layers: feature (convolutions) and classification (dense)
feature_layers = [
    layers.Conv2D(filters, kernel_size,
                  padding='valid',
                  input_shape=input_shape),
    layers.Activation('relu'),
    layers.Conv2D(filters, kernel_size),
    layers.Activation('relu'),
    layers.MaxPooling2D(pool_size=pool_size),
    layers.Dropout(0.25),
    layers.Flatten(),
]

classification_layers = [
    layers.Dense(128),
    layers.Activation('relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes),
    layers.Activation('softmax')
]

In [6]:
# create complete model
model = models.Sequential(feature_layers + classification_layers)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
activation_1 (Activation)    (None, 26, 26, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 24, 24, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 12, 12, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)  

In [7]:
def train_model(model, train, test, num_classes):
    x_train = train[0].reshape((train[0].shape[0],) + input_shape)
    x_test = test[0].reshape((test[0].shape[0],) + input_shape)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    y_train = keras.utils.to_categorical(train[1], num_classes)
    y_test = keras.utils.to_categorical(test[1], num_classes)

    model.compile(loss='categorical_crossentropy',
                  optimizer='adadelta',
                  metrics=['accuracy'])

    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test, y_test))
    score = model.evaluate(x_test, y_test, verbose=0)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])

In [8]:
# train model for 5-digit classification [0..4]
train_model(model,
            (x_train_lt5, y_train_lt5),
            (x_test_lt5, y_test_lt5), num_classes)

x_train shape: (30596, 28, 28, 1)
30596 train samples
5139 test samples
Instructions for updating:
Use tf.cast instead.
Train on 30596 samples, validate on 5139 samples
Epoch 1/5
30596/30596 [==============================] - 4s 122us/step - loss: 0.1632 - acc: 0.9482 - val_loss: 0.0293 - val_acc: 0.9897
Epoch 2/5
30596/30596 [==============================] - 2s 52us/step - loss: 0.0463 - acc: 0.9862 - val_loss: 0.0131 - val_acc: 0.9961
Epoch 3/5
30596/30596 [==============================] - 2s 53us/step - loss: 0.0326 - acc: 0.9903 - val_loss: 0.0107 - val_acc: 0.9961
Epoch 4/5
30596/30596 [==============================] - 2s 52us/step - loss: 0.0255 - acc: 0.9920 - val_loss: 0.0109 - val_acc: 0.9957
Epoch 5/5
30596/30596 [==============================] - 2s 52us/step - loss: 0.0218 - acc: 0.9941 - val_loss: 0.0086 - val_acc: 0.9969
Test score: 0.00858409407572483
Test accuracy: 0.9968865538042421


In [9]:
# freeze feature layers and rebuild model
for l in feature_layers:
    l.trainable = False

# transfer: train dense layers for new classification task [5..9]
train_model(model,
            (x_train_gte5, y_train_gte5),
            (x_test_gte5, y_test_gte5), num_classes)

x_train shape: (29404, 28, 28, 1)
29404 train samples
4861 test samples
Train on 29404 samples, validate on 4861 samples
Epoch 1/5
29404/29404 [==============================] - 1s 48us/step - loss: 0.2416 - acc: 0.9317 - val_loss: 0.0520 - val_acc: 0.9823
Epoch 2/5
29404/29404 [==============================] - 1s 40us/step - loss: 0.0810 - acc: 0.9752 - val_loss: 0.0391 - val_acc: 0.9856
Epoch 3/5
29404/29404 [==============================] - 1s 38us/step - loss: 0.0618 - acc: 0.9811 - val_loss: 0.0325 - val_acc: 0.9897
Epoch 4/5
29404/29404 [==============================] - 1s 40us/step - loss: 0.0518 - acc: 0.9844 - val_loss: 0.0258 - val_acc: 0.9916
Epoch 5/5
29404/29404 [==============================] - 1s 39us/step - loss: 0.0440 - acc: 0.9865 - val_loss: 0.0247 - val_acc: 0.9914
Test score: 0.024743171136617826
Test accuracy: 0.9913598025097716
